##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# トレーニングのチェックポイント

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/checkpoint"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
  <td>Google Colab で実行</td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/guide/checkpoint.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/guide/checkpoint.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>

「TensorFlow のモデルを保存する」という言いまわしは通常、次の 2 つのいずれかを意味します。

1. チェックポイント、または
2. 保存されたモデル（SavedModel）

チェックポイントは、モデルで使用されるすべてのパラメータ（`tf.Variable`オブジェクト）の正確な値をキャプチャします。チェックポイントにはモデルで定義された計算のいかなる記述も含まれていないため、通常は、保存されたパラメータ値を使用するソースコードが利用可能な場合に限り有用です。

一方、SavedModel 形式には、パラメータ値（チェックポイント）に加え、モデルで定義された計算のシリアライズされた記述が含まれています。この形式のモデルは、モデルを作成したソースコードから独立しています。したがって、TensorFlow Serving、TensorFlow Lite、TensorFlow.js、または他のプログラミング言語のプログラム（C、C++、Java、Go、Rust、C# などの TensorFlow API）を介したデプロイに適しています。

このガイドでは、チェックポイントの書き込みと読み取りを行う API について説明します。

## セットアップ

In [ ]:
import tensorflow as tf

In [ ]:
class Net(tf.keras.Model):
  """A simple linear model."""

  def __init__(self):
    super(Net, self).__init__()
    self.l1 = tf.keras.layers.Dense(5)

  def call(self, x):
    return self.l1(x)

In [ ]:
net = Net()

## `tf.keras`トレーニング API から保存する

[`tf.keras`の保存と復元に関するガイド](./keras/overview.ipynb#save_and_restore)をご覧ください。

`tf.keras.Model.save_weights`で TensorFlow チェックポイントを保存します。 

In [ ]:
net.save_weights('easy_checkpoint')

## チェックポイントを記述する


TensorFlow モデルの永続的な状態は、`tf.Variable`オブジェクトに格納されます。これらは直接作成できますが、多くの場合は`tf.keras.layers`や`tf.keras.Model`などの高レベル API を介して作成されます。

変数を管理する最も簡単な方法は、変数を Python オブジェクトにアタッチし、それらのオブジェクトを参照することです。

`tf.train.Checkpoint`、`tf.keras.layers.Layer`および`tf.keras.Model`のサブクラスは、属性に割り当てられた変数を自動的に追跡します。以下の例では、単純な線形モデルを作成し、モデルのすべての変数の値を含むチェックポイントを記述します。

`Model.save_weights`で、モデルチェックポイントを簡単に保存できます。

### 手動チェックポイント

#### セットアップ

`tf.train.Checkpoint` のすべての機能を実演するために、トイデータセットと最適化ステップを次のように定義します。

In [ ]:
def toy_dataset():
  inputs = tf.range(10.)[:, None]
  labels = inputs * 5. + tf.range(5.)[None, :]
  return tf.data.Dataset.from_tensor_slices(
    dict(x=inputs, y=labels)).repeat().batch(2)

In [ ]:
def train_step(net, example, optimizer):
  """Trains `net` on `example` using `optimizer`."""
  with tf.GradientTape() as tape:
    output = net(example['x'])
    loss = tf.reduce_mean(tf.abs(output - example['y']))
  variables = net.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return loss

#### チェックポイントオブジェクトを作成する

チェックポイントを手動で作成するには、`tf.train.Checkpoint` オブジェクトを使用します。チェックポイントを設定するオブジェクトは、オブジェクトの属性として設定されます。

`tf.train.CheckpointManager`は、複数のチェックポイントの管理にも役立ちます。

In [ ]:
opt = tf.keras.optimizers.Adam(0.1)
dataset = toy_dataset()
iterator = iter(dataset)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net, iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

#### モデルをトレーニングおよびチェックポイントする

次のトレーニングループは、モデルとオプティマイザのインスタンスを作成し、それらを`tf.train.Checkpoint`オブジェクトに集めます。それはデータの各バッチのループ内でトレーニングステップを呼び出し、定期的にチェックポイントをディスクに書き込みます。

In [ ]:
def train_and_checkpoint(net, manager):
  ckpt.restore(manager.latest_checkpoint)
  if manager.latest_checkpoint:
    print("Restored from {}".format(manager.latest_checkpoint))
  else:
    print("Initializing from scratch.")

  for _ in range(50):
    example = next(iterator)
    loss = train_step(net, example, opt)
    ckpt.step.assign_add(1)
    if int(ckpt.step) % 10 == 0:
      save_path = manager.save()
      print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))
      print("loss {:1.2f}".format(loss.numpy()))

In [ ]:
train_and_checkpoint(net, manager)

#### 復元してトレーニングを続ける

最初のトレーニングサイクルの後、新しいモデルとマネージャーを渡すことができますが、トレーニングはやめた所から再開します。

In [ ]:
opt = tf.keras.optimizers.Adam(0.1)
net = Net()
dataset = toy_dataset()
iterator = iter(dataset)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net, iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

train_and_checkpoint(net, manager)

`tf.train.CheckpointManager`オブジェクトは古いチェックポイントを削除します。上記では、最新の 3 つのチェックポイントのみを保持するように構成されています。

In [ ]:
print(manager.checkpoints)  # List the three remaining checkpoints

これらのパス、例えば`'./tf_ckpts/ckpt-10'`などは、ディスク上のファイルではなく、`index`ファイルのプレフィックスで、変数値を含む 1 つまたはそれ以上のデータファイルです。これらのプレフィックスは、まとめて単一の`checkpoint`ファイル(`'./tf_ckpts/checkpoint'`)にグループ化され、`CheckpointManager`がその状態を保存します。

In [ ]:
!ls ./tf_ckpts

<a id="loading_mechanics"></a>

## 読み込みの仕組み

TensorFlowは、読み込まれたオブジェクトから始めて、名前付きエッジを持つ有向グラフを走査することにより、変数をチェックポイントされた値に合わせます。エッジ名は通常、オブジェクトの属性名に由来しており、`self.l1 = tf.keras.layers.Dense(5)`の`"l1"`などがその例です。`tf.train.Checkpoint`は、`tf.train.Checkpoint(step=...)`の`"step"`のように、キーワード引数名を使用します。

上記の例の依存関係グラフは次のようになります。

![Visualization of the dependency graph for the example training loop](https://tensorflow.org/images/guide/whole_checkpoint.svg)

オプティマイザは赤、通常の変数は青、オプティマイザスロット変数はオレンジで表されています。`tf.train.Checkpoint` を表すノードなどは黒で示されています。

Slot variables are part of the optimizer's state, but are created for a specific variable. For example, the `'m'` edges above correspond to momentum, which the Adam optimizer tracks for each variable. Slot variables are only saved in a checkpoint if the variable and the optimizer would both be saved, thus the dashed edges.

`tf.train.Checkpoint` オブジェクトで `restore` を読み出すと、リクエストされた復元がキューに入れられ、`Checkpoint` オブジェクトから一致するパスが見つかるとすぐに変数値が復元されます。たとえば、ネットワークとレイヤーを介してバイアスのパスを再構築すると、上記で定義したモデルからそのバイアスのみを読み込むことができます。

In [ ]:
to_restore = tf.Variable(tf.zeros([5]))
print(to_restore.numpy())  # All zeros
fake_layer = tf.train.Checkpoint(bias=to_restore)
fake_net = tf.train.Checkpoint(l1=fake_layer)
new_root = tf.train.Checkpoint(net=fake_net)
status = new_root.restore(tf.train.latest_checkpoint('./tf_ckpts/'))
print(to_restore.numpy())  # We get the restored value now

これらの新しいオブジェクトの依存関係グラフは、上で書いたより大きなチェックポイントのはるかに小さなサブグラフです。 これには、バイアスと `tf.train.Checkpoint` がチェックポイントに番号付けするために使用する保存カウンタのみが含まれます。

![Visualization of a subgraph for the bias variable](https://tensorflow.org/images/guide/partial_checkpoint.svg)

`restore` は、オプションのアサーションを持つステータスオブジェクトを返します。新しい `Checkpoint` で作成されたすべてのオブジェクトが復元されるため、`status.assert_existing_objects_matched` がパスとなります。

In [ ]:
status.assert_existing_objects_matched()

チェックポイントには、レイヤーのカーネルやオプティマイザの変数など、一致しない多くのオブジェクトがあります。`status.assert_consumed()` は、チェックポイントとプログラムが正確に一致する場合に限りパスするため、ここでは例外がスローされます。

### Deferred restorations

TensorFlow の`Layer`オブジェクトは、入力形状が利用可能な場合、最初の呼び出しまで変数の作成を遅らせる可能性があります。例えば、`Dense`レイヤーのカーネルの形状はレイヤーの入力形状と出力形状の両方に依存するため、コンストラクタ引数として必要な出力形状は、単独で変数を作成するために充分な情報ではありません。`Layer`の呼び出しは変数の値も読み取るため、復元は変数の作成とその最初の使用の間で発生する必要があります。

To support this idiom, `tf.train.Checkpoint` defers restores which don't yet have a matching variable.

In [ ]:
deferred_restore = tf.Variable(tf.zeros([1, 5]))
print(deferred_restore.numpy())  # Not restored; still zeros
fake_layer.kernel = deferred_restore
print(deferred_restore.numpy())  # Restored

### チェックポイントを手動で検査する

`tf.train.load_checkpoint` は、チェックポイントのコンテンツにより低いレベルのアクセスを提供する `CheckpointReader` を返します。これには各変数のキーからチェックポイントの各変数の形状と dtype へのマッピングが含まれます。変数のキーは上に表示されるグラフのようなオブジェクトパスです。

注意: チェックポイントへのより高いレベルの構造はありません。変数のパスと値のみが認識されており、`models`、`layers`、またはそれらがどのように接続されているかについての概念が一切ありません。

In [ ]:
tf.train.list_variables(tf.train.latest_checkpoint('./tf_ckpts/'))

`net.l1.kernel` の値に関心がある場合は、次のコードを使って値を取得できます。

In [ ]:
key = 'net/l1/kernel/.ATTRIBUTES/VARIABLE_VALUE'

print("Shape:", shape_from_key[key])
print("Dtype:", dtype_from_key[key].name)

また、変数の値を検査できるようにする `get_tensor` メソッドも提供されています。

In [ ]:
reader.get_tensor(key)

### Object tracking

`self.l1 = tf.keras.layers.Dense(5)`のような直接の属性割り当てと同様に、リストとディクショナリを属性に割り当てると、それらの内容を追跡します。

`self.l1 = tf.keras.layers.Dense(5)`のような直接の属性割り当てと同様に、リストとディクショナリを属性に割り当てると、それらの内容を追跡します。

In [ ]:
save = tf.train.Checkpoint()
save.listed = [tf.Variable(1.)]
save.listed.append(tf.Variable(2.))
save.mapped = {'one': save.listed[0]}
save.mapped['two'] = save.listed[1]
save_path = save.save('./tf_list_example')

restore = tf.train.Checkpoint()
v2 = tf.Variable(0.)
assert 0. == v2.numpy()  # Not restored yet
restore.mapped = {'two': v2}
restore.restore(save_path)
assert 2. == v2.numpy()

リストとディクショナリのラッパーオブジェクトにお気づきでしょうか。これらのラッパーは基礎的なデータ構造のチェックポイント可能なバージョンです。属性に基づく読み込みと同様に、これらのラッパーは変数の値がコンテナに追加されるとすぐにそれを復元します。

In [ ]:
restore.listed = []
print(restore.listed)  # ListWrapper([])
v1 = tf.Variable(0.)
restore.listed.append(v1)  # Restores v1, from restore() in the previous cell
assert 1. == v1.numpy()

Trackable objects include `tf.train.Checkpoint`, `tf.Module` and its subclasses (e.g. `keras.layers.Layer` and `keras.Model`), and recognized Python containers:

 * `dict` (and `collections.OrderedDict`)
 * `list`
 * `tuple` (and `collections.namedtuple`, `typing.NamedTuple`)

Other container types are **not supported**, including:

 * `collections.defaultdict`
 * `set`

All other Python objects are **ignored**, including:

 * `int`
 * `string`
 * `float`


## まとめ

TensorFlow オブジェクトは、それらが使用する変数の値を保存および復元するための容易で自動的な仕組みを提供します。
